# 미션 소개

Hugging Face transformers 라이브러리를 사용하여 문서 요약 모델을 구현하는 미션.
데이터 로드 및 전처리부터 모델 실행, 결과 평가까지 전체 파이프라인을 구축.

## 사용 데이터셋
- 데이터 형식
    - JSON 파일 형태로 제공되며, 3종류(신문 기사, 사설, 법률)의 문서가 포함되어 있다.
- 데이터 구성
    - 각 문서 타입은 train/test 쌍으로 구성되어 있으며, 전체 데이터를 모두 사용하거나 원하는 문서 종류를 선택하여 학습시키면 된다.

## 가이드라인
- 데이터 로드 및 전처리
    - 문서 데이터를 로드하고, 불필요한 기호나 공백을 제거하는 등 전처리 작업을 수행
    - 텍스트 길이를 확인하고, 모델 입력에 적합한 형식으로 변환한다.
- 모델 선택 및 실행
    - Hugging Face의 Transformers 라이브러리를 활용해 문서 요약을 수행
    - 사전 학습된 모델을 활용하거나 주어진 데이터를 가지고 Fine-tuning 하기.
- 모델 평가 및 결과 분석
    - 생성된 요약문과 원본 문서를 비교하여 ROUGE 등의 평가 지표를 사용해 요약 품질을 분석한다.
    - 테스트 문장에 대한 요약 결과를 출력하여 모델의 성능을 확인한다.
- 모델 구현 및 학습 결과
    - 문서 요약 모델(예: Transformer 기반 요약 모델, T5, BART 등)을 구현하고, 데이터 로드 → 전처리 → 모델 구축 및 학습 → 요약 생성 및 평가 과정을 순차적으로 진행.
- 모델 성능 평가 및 제출
    - 생성된 요약문의 품질을 정성적(요약 결과 확인) 및 정량적(ROUGE 등)으로 평가.
    - 테스트 데이터셋에 대한 요약 결과를 포함
- 원본 데이터셋 링크
    - https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=97

# 환경 설정

In [1]:
!pip install transformers datasets
!pip install konlpy

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached xxhash-3.5.0-cp312-cp312-macosx_11_0_arm64.whl (30 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [datasets]4/5 [datasets]


In [2]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import random
import pickle
from tqdm import tqdm
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from konlpy.tag import Okt
import nltk
from nltk.tokenize import word_tokenize

## GPU 세팅

In [3]:
print("PyTorch:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
print("CUDA available:", torch.cuda.is_available())

if torch.backends.mps.is_available():
    device = torch.device("mps")  # 맥북 M1/M2 GPU
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device("cuda")  # NVIDIA GPU (Colab, Windows 등)
    print("Using CUDA (NVIDIA GPU)")
else:
    device = torch.device("cpu")   # CPU fallback
    print("Using CPU")

print("Selected device:", device)

PyTorch: 2.8.0
MPS available: True
CUDA available: False
Using MPS (Apple Silicon GPU)
Selected device: mps


# KoBART 모델

In [4]:
# KoBART 모델 다운로드
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

print("=== gogamza/kobart-base-v2 모델 다운로드 중 ===")

# 토크나이저 다운로드 (models 폴더에 저장)
tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2', cache_dir='./models')
print("✅ 토크나이저 다운로드 완료!")

# 모델 다운로드 (models 폴더에 저장)
model = AutoModelForSeq2SeqLM.from_pretrained('gogamza/kobart-base-v2', cache_dir='./models')
print("✅ 모델 다운로드 완료!")

# 모델 정보 출력
print(f"\n�� 모델 정보:")
print(f"- 토크나이저 타입: {type(tokenizer).__name__}")
print(f"- 모델 타입: {type(model).__name__}")
print(f"- 어휘 크기: {tokenizer.vocab_size:,}")
print(f"- 모델 파라미터: {sum(p.numel() for p in model.parameters()):,}")

# 모델을 디바이스로 이동
model = model.to(device)
print(f"\n�� 모델이 {device} 디바이스로 이동되었습니다!")

=== gogamza/kobart-base-v2 모델 다운로드 중 ===


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


✅ 토크나이저 다운로드 완료!


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


✅ 모델 다운로드 완료!

�� 모델 정보:
- 토크나이저 타입: PreTrainedTokenizerFast
- 모델 타입: BartForConditionalGeneration
- 어휘 크기: 30,000
- 모델 파라미터: 123,859,968

�� 모델이 mps 디바이스로 이동되었습니다!


# 데이터 로드 및 전처리

## 1. 데이터 로드 및 전처리 함수

In [8]:
# 데이터 로드 및 전처리 함수
def load_json_dataset(file_path):
    """JSON 파일을 로드하여 문서별 text, summary 정보를 추출"""
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    examples = []
    for doc in data["documents"]:
        sentences = []
        # "text"는 중첩 리스트 형태이므로 내부의 모든 sentence를 추출
        for sublist in doc["text"]:
            for item in sublist:
                sentences.append(item.get("sentence", ""))
        
        full_text = " ".join(sentences)
        # abstractive 요약은 첫번째 항목 사용 (없으면 빈 문자열)
        summary = doc["abstractive"][0] if doc["abstractive"] else ""
        
        examples.append({
            "text": full_text,
            "summary": summary,
        })
    
    return examples

## 2. 데이터셋 로드

In [9]:
# 데이터 경로 설정
base_path = "./summarization/"

# 처음에는 작은 법률 데이터셋으로 시작
train_file = "train_original_law.json"
valid_file = "valid_original_law.json"

print("=== 데이터 로드 중 ===")
train_examples = load_json_dataset(os.path.join(base_path, train_file))
valid_examples = load_json_dataset(os.path.join(base_path, valid_file))

print(f"훈련 데이터: {len(train_examples):,}개")
print(f"검증 데이터: {len(valid_examples):,}개")

# 샘플 데이터 확인
print(f"\n=== 샘플 데이터 ===")
print(f"첫 번째 훈련 예시:")
print(f"텍스트 길이: {len(train_examples[0]['text'])}자")
print(f"요약 길이: {len(train_examples[0]['summary'])}자")
print(f"텍스트 미리보기: {train_examples[0]['text'][:100]}...")
print(f"요약: {train_examples[0]['summary']}")

=== 데이터 로드 중 ===
훈련 데이터: 24,329개
검증 데이터: 3,004개

=== 샘플 데이터 ===
첫 번째 훈련 예시:
텍스트 길이: 372자
요약 길이: 97자
텍스트 미리보기: 원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고, 노조조합장이 사임한 경우, 노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있...
요약: 원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.


## 3. 데이터 전처리

In [10]:
# 데이터 전처리 함수
def preprocess_data(examples, max_text_length=512, max_summary_length=128):
    """텍스트와 요약을 전처리하고 길이 제한"""
    processed = []
    
    for example in examples:
        text = example["text"].strip()
        summary = example["summary"].strip()
        
        # 빈 요약 제거
        if not summary:
            continue
            
        # 길이 제한
        if len(text) > max_text_length * 3:  # 대략적인 토큰 수 추정
            continue
        if len(summary) > max_summary_length * 3:
            continue
            
        processed.append({
            "text": text,
            "summary": summary
        })
    
    return processed

# 데이터 전처리 적용
print("=== 데이터 전처리 중 ===")
train_processed = preprocess_data(train_examples)
valid_processed = preprocess_data(valid_examples)

print(f"전처리 후 훈련 데이터: {len(train_processed):,}개")
print(f"전처리 후 검증 데이터: {len(valid_processed):,}개")

=== 데이터 전처리 중 ===
전처리 후 훈련 데이터: 22,514개
전처리 후 검증 데이터: 2,836개


## 4. Hugging Face Dataset으로 변환

In [11]:
from datasets import Dataset, DatasetDict

# Dataset 객체 생성
train_dataset = Dataset.from_list(train_processed)
valid_dataset = Dataset.from_list(valid_processed)

# DatasetDict 형태로 통합
dataset = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset
})

print("=== Dataset 변환 완료 ===")
print(f"데이터셋 구조: {dataset}")

=== Dataset 변환 완료 ===
데이터셋 구조: DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 22514
    })
    validation: Dataset({
        features: ['text', 'summary'],
        num_rows: 2836
    })
})


## 5. tokenizing

In [13]:
# 토크나이징 함수
def tokenize_function(example):
    """텍스트와 요약을 토큰화"""
    model_inputs = tokenizer(
        example["text"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    
    labels = tokenizer(
        text_target=example["summary"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 토크나이징 적용
print("=== 토크나이징 중 ===")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

print(f"✅ 토크나이징 완료!")
print(f"토크나이징된 훈련 데이터: {len(tokenized_datasets['train'])}개")
print(f"토크나이징된 검증 데이터: {len(tokenized_datasets['validation'])}개")

=== 토크나이징 중 ===


Map:   0%|          | 0/22514 [00:00<?, ? examples/s]

Map:   0%|          | 0/2836 [00:00<?, ? examples/s]

✅ 토크나이징 완료!
토크나이징된 훈련 데이터: 22514개
토크나이징된 검증 데이터: 2836개


# 모델 학습

## DataCollator 설정

In [14]:
# DataCollator 설정
from transformers import DataCollatorForSeq2Seq

print("=== DataCollator 설정 중 ===")
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)

print("✅ DataCollator 설정 완료!")

=== DataCollator 설정 중 ===
✅ DataCollator 설정 완료!


## 학습 파라미터 설정

In [16]:
# 학습 파라미터 설정
from transformers import TrainingArguments

print("=== 학습 파라미터 설정 중 ===")

training_args = TrainingArguments(
    output_dir="./results",           # 결과 저장 폴더
    eval_strategy="steps",            # 평가 전략
    eval_steps=500,                   # 500 스텝마다 평가
    save_strategy="steps",            # 저장 전략
    save_steps=1000,                  # 1000 스텝마다 모델 저장
    learning_rate=5e-5,               # 학습률
    per_device_train_batch_size=4,    # 배치 크기 (GPU 메모리에 따라 조정)
    per_device_eval_batch_size=4,     # 평가 배치 크기
    num_train_epochs=3,               # 학습 에포크
    weight_decay=0.01,                # 가중치 감쇠
    logging_dir="./logs",             # 로그 저장 폴더
    logging_steps=100,                # 100 스텝마다 로그
    save_total_limit=3,               # 최대 3개 체크포인트만 저장
    load_best_model_at_end=True,      # 최고 성능 모델 로드
    metric_for_best_model="eval_loss", # 최고 성능 기준
    greater_is_better=False,          # 손실은 낮을수록 좋음
    report_to="none",                 # wandb 등 외부 도구 사용 안함
)

print("✅ 학습 파라미터 설정 완료!")
print(f"학습 에포크: {training_args.num_train_epochs}")
print(f"학습률: {training_args.learning_rate}")
print(f"배치 크기: {training_args.per_device_train_batch_size}")

=== 학습 파라미터 설정 중 ===
✅ 학습 파라미터 설정 완료!
학습 에포크: 3
학습률: 5e-05
배치 크기: 4


## Trainer 설정 및 학습

In [17]:
# Trainer 설정
from transformers import Trainer

print("=== Trainer 설정 중 ===")

trainer = Trainer(
    model=model,                           # 모델
    args=training_args,                    # 학습 파라미터
    train_dataset=tokenized_datasets["train"],      # 훈련 데이터
    eval_dataset=tokenized_datasets["validation"],  # 검증 데이터
    tokenizer=tokenizer,                   # 토크나이저
    data_collator=data_collator,           # 데이터 콜레이터
)

print("✅ Trainer 설정 완료!")
print(f"훈련 데이터 크기: {len(tokenized_datasets['train'])}")
print(f"검증 데이터 크기: {len(tokenized_datasets['validation'])}")

=== Trainer 설정 중 ===
✅ Trainer 설정 완료!
훈련 데이터 크기: 22514
검증 데이터 크기: 2836


/var/folders/mv/xvtnn0v91cx8bl846n_qn4100000gn/T/ipykernel_87676/3876579206.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
# 모델 학습 시작
print("🚀 모델 학습 시작!")
print("=" * 50)

# 학습 실행
train_result = trainer.train()

print("✅ 학습 완료!")
print(f"총 학습 시간: {train_result.metrics['train_runtime']:.2f}초")
print(f"총 학습 스텝: {train_result.metrics['train_steps']}")
print(f"최종 손실: {train_result.metrics['train_loss']:.4f}")

🚀 모델 학습 시작!


/Users/leeyoungho/miniforge3/envs/ai_3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss
500,0.893400,0.752535
1000,0.817400,0.704269


/Users/leeyoungho/miniforge3/envs/ai_3/lib/python3.12/site-packages/transformers/modeling_utils.py:3909: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


KeyboardInterrupt: 

# 요약 생성